## Ejercicio 4 - Documentando noticias locales y en el extranjero
Eligan dos sitios web de noticias: uno de Argentina y uno del extranjero (puede ser en castellano o en inglés como el *New York Times*).

Utilicen herramientas de webscraping para obtener los **links** de las noticias de la portada en ambos diarios. Guarden cada titular con los links obtenidos en un dataframe. Por lo tanto, debe quedarles un dataframe de 4 columnas: una columna para noticias locales, otra para su link, otra columna de noticias extranjeras y una ultima columna de sus links. Luego, expórtenlo a un archivo de excel.

En el reporte este esta parte B:
1) Inserten una captura de pantalla de la pagina del diario extranjero y en Argentina de las cuales hacen el ejercicio de extraer titulares y sus links correspondientes. Esto servirá al momento de la corrección para verificar que los links obtenidos hacen referencia a las noticias de ese día y hora.
2) Comenten brevemente (1 párrafo) las diferencias/similitudes al extraer titulares en diarios de Argentina versus diario extranjero
3) Comenten brevemente (1 párrafo) las dificultades y cómo resolvieron dichas dificultades.

*Nota*: es posible que logren obtener los links a las noticias sin el dominio: por ejemplo "https://www.lanacion.com.ar/". De ser así, concatenen el dominio a la ruta del link obtenido, tal que se obtenga un link al que se pueda acceder. Es decir, que las cadenas de caracteres finales tendrán la forma: https://www.lanacion.com.ar/*texto_obtenido*)

### Preparación de paquetes

In [6]:
#Importamos los paquetes relevantes
import pandas as pd # manipular DF's
import requests # HTTP requestor
from bs4 import BeautifulSoup # HTML parser
print("hola") # Pruebo que el kernel esté funcionando

hola


### Scraping de diario extranjero

In [7]:
# Presentamos las URLs y su contenido HTML
# urlarg = INSERTAR LINK
urlext = "https://www.washingtonpost.com/" #URL Extranjera

In [8]:
#Código para imprimir el HTML
def imprimirHTML(link):
    rext = requests.get(link)
    contenido_ext = rext.content
    soupext = BeautifulSoup(contenido_ext,"html.parser")
    print(soupext)

In [9]:
# imprimirHTML("https://www.washingtonpost.com/") # Si bien definí el link más arriba con la variable 'urlext', defino una función para probar varios links más rápido

Por algún motivo, la ejecución de la "sopa" no imprime nada, pero tampoco devuelve error. Sospecho que puede ser algo propio de Washington Post, como si tuviera algún tipo de protección.

In [10]:
# Mejoro la función creada para que reporte error de acceso si es que existe:
def imprimirHTML(link):
    try:
        response = requests.get(link)
        response.raise_for_status()  # Lanza error si hay problema HTTP
        contenido = response.content
        soup = BeautifulSoup(contenido, "html.parser")
        print(soup)
    except requests.exceptions.RequestException as e:
        print(f"Error al acceder a {link}: {e}")

# imprimirHTML("https://www.washingtonpost.com/")

Tampoco avanza, e incluso un simple 'print("hola")' queda sin cargare: debe ser un tema de permisos del Washington Post.
<br>Comentario adicional: al reiniciar el Kernel, el parseo del HTML del Washington Post devolvió un 'TimeoutError'.

Pruebo la función con un diario de menor relevancia:

In [ ]:
imprimirHTML("https://www.nzherald.co.nz/sport/rugby/") # New Zealand Rugby Herald

Funciona! Así que vamos a intentar parsear el HTML del Washington Post manualmente.

### Parseo del HTML (vía manual)

In [19]:
# 1. Copio el HTML completo del Washington Post desde la Web
# 2. Abro un bloc de notas en la misma ubicación de este notebook, pego el HTML y lo guardo como archivo .html
# 3. Leo el HTML desde este notebook

with open("washington_post.html", "r", encoding="utf-8") as f:
    html_doc = f.read()

print(html_doc[:500])  # muestra solo los primeros 500 caracteres

<html lang="en" data-scrim-state="closed"><head><link rel="icon" href="https://www.washingtonpost.com/favicon.svg" type="image/svg+xml"><link rel="icon" href="https://www.washingtonpost.com/touch-icon-iphone.png" type="image/png"><link rel="apple-touch-icon" href="https://www.washingtonpost.com/touch-icon-iphone.png"><link rel="apple-touch-icon" sizes="152x152" href="https://www.washingtonpost.com/touch-icon-ipad.png"><link rel="apple-touch-icon" sizes="167x167" href="https://www.washingtonpost.


In [18]:
# Pruebo manualmente el primer H2 de la web:

<h2 class="wpds-c-iiQaMf wpds-c-iiQaMf-ibUhiPg-css"><a data-pb-local-content-field="web_headline" href="https://www.washingtonpost.com/politics/2025/09/04/tariffs-trump-supreme-court-appeal/"><span>Trump officials ask Supreme Court to quickly allow sweeping tariffs</span></a></h2>

La descripción de la 'class' del H2 traído es: "wpds-c-iiQaMf wpds-c-iiQaMf-ibUhiPg-css"

Ahora, el objetivo es armar un DataFrame de 2 columnas:
 - Columna con los títulos de la página de inicio
 - Columna con los respectivos links a las notas completas de dichos títulos

### Traer todos los elementos H2 del HTML
(Se trae el código crudo y después se lo limpia, para quedarnos sólo con los títulos, los links y así armar el DataFrame)

In [22]:
# 1. Leer el HTML desde tu archivo
with open("washington_post.html", "r", encoding="utf-8") as f:
    contenido = f.read()

# 2. Crear el objeto BeautifulSoup
soup = BeautifulSoup(contenido, "html.parser")

# 3. Buscar todos los elementos <h2>
h2_elements = soup.find_all("h2")

# 4. Mostrar el resultado
print("\n", h2_elements)


 [<h2 class="wpds-c-iiQaMf wpds-c-iiQaMf-ibUhiPg-css"><a data-pb-local-content-field="web_headline" href="https://www.washingtonpost.com/health/2025/09/04/coronavirus-vaccines-mandates-state-divisions/"><span>RFK Jr. drives a wedge between red and blue states on vaccines</span></a></h2>, <h2 class="wpds-c-iiQaMf wpds-c-iiQaMf-ihMULTC-css"><a data-pb-local-content-field="web_headline" href="https://www.washingtonpost.com/health/2025/09/04/rfk-vaccines-cdc-hearing/"><span>Kennedy to be grilled on vaccine policy after firing of CDC director</span></a></h2>, <h2 class="wpds-c-iiQaMf wpds-c-iiQaMf-iiBDqjX-css"><a data-pb-local-content-field="web_headline" href="https://www.washingtonpost.com/politics/2025/09/04/tariffs-trump-supreme-court-appeal/"><span>Trump officials ask Supreme Court to quickly allow sweeping tariffs</span></a></h2>, <h2 class="wpds-c-iiQaMf wpds-c-iiQaMf-ihMULTC-css"><a data-pb-local-content-field="web_headline" href="https://www.washingtonpost.com/business/2025/09/04/

In [54]:
titulos = []
links = []

for h2 in h2_elements:
    h2_text = h2.text.strip()
    a_tag = h2.find("a")
    if a_tag:
        h2_link = a_tag.get('href')
    else:
        h2_link = None
    titulos.append(h2_text)
    links.append(h2_link)

dfext = pd.DataFrame({"Titulo": titulos, "Link": links})
dfext

,Titulo,Link
0,RFK Jr. drives a wedge between red and blue st...,https://www.washingtonpost.com/health/2025/09/...
1,Kennedy to be grilled on vaccine policy after ...,https://www.washingtonpost.com/health/2025/09/...
2,Trump officials ask Supreme Court to quickly a...,https://www.washingtonpost.com/politics/2025/0...
3,See where tariffs are driving up prices on hou...,https://www.washingtonpost.com/business/2025/0...
4,Inside the Trump team’s conflicting efforts to...,https://www.washingtonpost.com/world/2025/09/0...
5,The pigeon heist: How million-dollar racing bi...,https://www.washingtonpost.com/world/interacti...
6,House GOP weighing bills to remove elected D.C...,https://www.washingtonpost.com/dc-md-va/2025/0...
7,"At D.C. Superior Court, a system up at all hou...",https://www.washingtonpost.com/dc-md-va/2025/0...
8,National Guard deployment in D.C. expected to ...,https://www.washingtonpost.com/national-securi...
9,Trump floats sending law enforcement to New Or...,https://www.washingtonpost.com/politics/2025/0...


Como no puedo ver los links completos en el output del DF, los desempaco y reviso si son funcionales.

In [60]:
dfext['Link'].to_list() # Sin el Método to_list(), el notebook imprime sólo una parte de los links, y de esa forma no funcionan

['https://www.washingtonpost.com/health/2025/09/04/coronavirus-vaccines-mandates-state-divisions/',
 'https://www.washingtonpost.com/health/2025/09/04/rfk-vaccines-cdc-hearing/',
 'https://www.washingtonpost.com/politics/2025/09/04/tariffs-trump-supreme-court-appeal/',
 'https://www.washingtonpost.com/business/2025/09/04/trump-tariffs-imports-products-prices/',
 'https://www.washingtonpost.com/world/2025/09/04/india-us-trump-modi-tariffs-trade/',
 'https://www.washingtonpost.com/world/interactive/2025/pigeon-racing-china-crime-belgium/',
 'https://www.washingtonpost.com/dc-md-va/2025/09/03/house-gop-weighing-bills-remove-elected-dc-ag-overhaul-justice-policies/',
 'https://www.washingtonpost.com/dc-md-va/2025/09/03/trump-dc-court-community-organizers/',
 'https://www.washingtonpost.com/national-security/2025/09/03/national-guard-dc/',
 'https://www.washingtonpost.com/politics/2025/09/03/trump-policing-red-states-new-orleans/',
 'https://www.washingtonpost.com/obituaries/2025/09/04/gior

Listo! Ya tenemos creado el DataFrame de los titulares extranjeros con los links funcionando correctamente.